In [19]:
import numpy as np

N = 20
t = (np.arange(1, N+1)) / N

# 1a. 生成一次“真实”的测量数据 ym
a_true = [10, 10, 2]
y_true = a_true[0] * t**2 * np.exp(-a_true[1]*t**a_true[2])
noise = 0.03
ym = y_true + noise * np.random.rand(N) # ym 现在是固定的

def f(vec):
    a1 = vec[0]
    a2 = vec[1]
    a3 = vec[2]
    
    y_model = a1 * t**2 * np.exp(-a2*t**a3)
    return y_model - ym

def fdjac(f, fx, x):
    m = len(fx)
    n = len(x)
    J = np.zeros((m, n))
    h = 1e-12
    
    for j in range(n):
        x_h = x.copy()
        x_h[j] = x_h[j] + h
        fx_h = f(x_h)

        J[:, j] = (fx_h - fx) / h

    return J

def levenberg(f, x1, maxiter = 50):
    n = len(x1)
    fk = f(x1)
    x = np.zeros((maxiter, n))
    x[0] = x1
    Ak = fdjac(f, fk, x1)

    k = 0
    lam = 10.0
    jac_is_new = True
    delta_p = 10.0
    
    while np.linalg.norm(fk) > 1e-12 and k < maxiter - 1 and np.linalg.norm(delta_p) > 1e-12:
        B = (Ak.T @ Ak) + lam * np.eye(n)
        z = - Ak.T @ fk

        delta_p = np.linalg.lstsq(B, z)[0]
        xnew = x[k] + delta_p
        fnew = f(xnew)
    
        if np.linalg.norm(fnew) < np.linalg.norm(fk):
            y = fnew - fk           # y_k = f_{k+1} - f_k
            x[k+1] = xnew         # 接受新点
            fk = fnew               # 更新
            Ak = Ak + np.outer(y - Ak @ delta_p, delta_p / np.dot(delta_p.T, delta_p))

            k += 1
            lam /= 10 #更激进 接近高斯牛顿法
            jac_is_new = False #标记Ak未重新更新 是由broyden迭代法迭代而来
        else:
            lam = lam * 4 #保守策略 退化为梯度法
            # 那么这个近似可能已经太差了，需要重新计算
            if not jac_is_new:
                # 支付高昂成本 重新计算一个新的 J
                Ak = fdjac(f, x[k], fk) 
                jac_is_new = True
    if np.linalg.norm(fk) > 1e-3:
        #如果迭代完成后还是很大 则说明未找到根
        print("Iteration did not find a root.")

    return x[k]
    #return x[:k+1]
a1 = np.array([10, 10, 2])
result = levenberg(f, a1)
print(result)    

Iteration did not find a root.
[10. 10.  2.]
